In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
from scipy.linalg import svd
from scipy.spatial.transform import Rotation
from scipy import signal
import io

# 全局变量，用于存储校准结果
calibration_results = {}

class IMUDataLoader:
    def __init__(self):
        self.static_data = None
        self.rotation_data = None
        self.sample_rate = 30.0  # Default sample rate 30Hz
        self.selected_device = None  # Selected device name
        
        # Filter settings
        self.filter_enabled = True
        self.filter_cutoff = 10.0  # Hz
        self.filter_order = 4
        
        # Updated column names for new data format (Chinese)
        self.accel_columns = ['加速度X(g)', '加速度Y(g)', '加速度Z(g)']  # Acceleration in g
        self.gyro_columns = ['角速度X(°/s)', '角速度Y(°/s)', '角速度Z(°/s)']  # Gyroscope columns
        self.device_column = '设备名称'  # Device name column
        
        # Conversion factor from g to m/s²
        self.g_to_ms2 = 9.80665
        
        # Available devices
        self.available_devices = []
        
    def apply_lowpass_filter(self, data, cutoff_freq, order):
        """Apply Butterworth lowpass filter to data"""
        if not self.filter_enabled or cutoff_freq >= self.sample_rate / 2:
            return data
            
        # Design Butterworth filter
        nyquist = self.sample_rate / 2
        normalized_cutoff = cutoff_freq / nyquist
        b, a = signal.butter(order, normalized_cutoff, btype='low', analog=False)
        
        # Apply filter to each column
        filtered_data = np.zeros_like(data)
        for i in range(data.shape[1]):
            # Apply zero-phase filtering (filtfilt) to avoid phase shift
            filtered_data[:, i] = signal.filtfilt(b, a, data[:, i])
        
        return filtered_data
        
    def detect_devices(self, df):
        """Detect available devices in the dataframe"""
        if self.device_column in df.columns:
            devices = df[self.device_column].dropna().unique()
            self.available_devices = list(devices)
            return self.available_devices
        return []
        
    def filter_by_device(self, df, device_name):
        """Filter dataframe by device name"""
        if self.device_column in df.columns and device_name:
            return df[df[self.device_column] == device_name].copy()
        return df
        
    def check_and_clean_data(self, df, data_type):
        """Check for NaN/Inf values and clean the data"""
        # Count NaN values in each column
        nan_counts = {}
        inf_counts = {}
        
        for col in self.accel_columns + self.gyro_columns:
            if col in df.columns:
                nan_counts[col] = df[col].isna().sum()
                inf_counts[col] = np.isinf(df[col]).sum()
        
        total_nans = sum(nan_counts.values())
        total_infs = sum(inf_counts.values())
        
        if total_nans > 0 or total_infs > 0:
            print(f"\n⚠️ Data quality issues detected in {data_type} data:")
            print(f"Total NaN values: {total_nans}")
            print(f"Total Inf values: {total_infs}")
            
            # First, replace inf with NaN
            for col in self.accel_columns + self.gyro_columns:
                if col in df.columns:
                    df.loc[np.isinf(df[col]), col] = np.nan
            
            # Then interpolate
            for col in self.accel_columns + self.gyro_columns:
                if col in df.columns:
                    # Use linear interpolation
                    df[col] = df[col].interpolate(method='linear', limit_direction='both')
                    
                    # If still have NaN at edges, use forward/backward fill
                    df[col] = df[col].fillna(method='ffill').fillna(method='bfill')
                    
                    # If still have NaN (all values were NaN), fill with 0
                    if df[col].isna().any():
                        print(f"  Warning: Column {col} has all NaN values, filling with 0")
                        df[col] = df[col].fillna(0)
            
            print(f"✅ Data cleaning successful!")
                
        return df
        
    def load_data(self, file_content, data_type):
        """Load IMU data file"""
        try:
            # Handle different file content types
            if isinstance(file_content, bytes):
                file_content_str = file_content.decode('utf-8')
                df = pd.read_csv(io.StringIO(file_content_str), sep='\t')
            elif isinstance(file_content, str):
                if os.path.exists(file_content):
                    df = pd.read_csv(file_content, sep='\t')
                else:
                    df = pd.read_csv(io.StringIO(file_content), sep='\t')
            else:
                df = pd.read_csv(io.BytesIO(file_content), sep='\t')
            
            # Strip whitespace from column names
            df.columns = df.columns.str.strip()
            
            # Detect available devices
            devices = self.detect_devices(df)
            if devices:
                print(f"Found devices: {devices}")
                if not self.selected_device and devices:
                    self.selected_device = devices[0]
                    print(f"Default selected device: {self.selected_device}")
            
            # Check for required columns
            missing_cols = []
            for col in self.accel_columns + self.gyro_columns:
                if col not in df.columns:
                    missing_cols.append(col)
            
            if missing_cols:
                raise ValueError(f"Missing required columns: {missing_cols}")
            
            # Clean the data
            df = self.check_and_clean_data(df, data_type)
            
            # Store the full dataframe
            if data_type == 'static':
                self.static_data = df
            elif data_type == 'rotation':
                self.rotation_data = df
            
            return df
            
        except Exception as e:
            print(f"Error loading data: {e}")
            return None
    
    def get_processed_data(self, data_type, apply_filter=True):
        """Get processed data for selected device with optional filtering"""
        if data_type == 'static' and self.static_data is not None:
            df = self.static_data
        elif data_type == 'rotation' and self.rotation_data is not None:
            df = self.rotation_data
        else:
            return None
            
        # Filter by selected device
        if self.selected_device:
            df = self.filter_by_device(df, self.selected_device)
            
        # Process the data
        df = df.copy()
        
        # Add timestamp
        num_samples = len(df)
        if num_samples > 0:
            df['Time'] = np.arange(0, num_samples / self.sample_rate, 1 / self.sample_rate)[:num_samples]
        
        # Apply lowpass filter if requested
        if apply_filter and self.filter_enabled:
            # Filter acceleration data
            accel_data = df[self.accel_columns].values
            filtered_accel = self.apply_lowpass_filter(accel_data, self.filter_cutoff, self.filter_order)
            df[self.accel_columns] = filtered_accel
            
            # Filter gyroscope data
            gyro_data = df[self.gyro_columns].values
            filtered_gyro = self.apply_lowpass_filter(gyro_data, self.filter_cutoff, self.filter_order)
            df[self.gyro_columns] = filtered_gyro
        
        return df
    
    def get_static_accel(self):
        """Get static acceleration data (with gravity, in m/s²)"""
        df = self.get_processed_data('static', apply_filter=True)
        if df is None or len(df) == 0:
            return None
        
        # Get acceleration data
        accel_data = df[self.accel_columns].values * self.g_to_ms2
        
        # Final check for NaN/Inf
        if np.any(np.isnan(accel_data)) or np.any(np.isinf(accel_data)):
            print("Warning: NaN or Inf values detected in static acceleration data")
            mask = ~(np.any(np.isnan(accel_data), axis=1) | np.any(np.isinf(accel_data), axis=1))
            accel_data = accel_data[mask]
            print(f"Removed {np.sum(~mask)} rows with bad data")
        
        return accel_data
    
    def get_rotation_gyro(self):
        """Get rotation gyroscope data (converted to rad/s)"""
        df = self.get_processed_data('rotation', apply_filter=True)
        if df is None or len(df) == 0:
            return None
        
        # Convert to rad/s
        gyro_data = df[self.gyro_columns].values * (np.pi / 180.0)
        
        # Final check for NaN/Inf
        if np.any(np.isnan(gyro_data)) or np.any(np.isinf(gyro_data)):
            print("Warning: NaN or Inf values detected in rotation gyroscope data")
            mask = ~(np.any(np.isnan(gyro_data), axis=1) | np.any(np.isinf(gyro_data), axis=1))
            gyro_data = gyro_data[mask]
            print(f"Removed {np.sum(~mask)} rows with bad data")
        
        return gyro_data
    
    def preview_data(self, data_type):
        """Preview acceleration and gyroscope data with filter comparison"""
        # Get both filtered and unfiltered data
        df_filtered = self.get_processed_data(data_type, apply_filter=True)
        df_unfiltered = self.get_processed_data(data_type, apply_filter=False)
        
        if df_filtered is None or len(df_filtered) == 0:
            print(f"No data available for {data_type}")
            return
        
        # Create figure with subplots
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        fig.suptitle(f'{data_type.capitalize()} Data Preview - {self.selected_device}', fontsize=14, weight='bold')
        
        time = df_filtered['Time'].values
        
        # Plot acceleration data (filtered vs unfiltered)
        colors = ['red', 'green', 'blue']
        labels = ['X', 'Y', 'Z']
        
        # Unfiltered acceleration
        for i, (col, color, label) in enumerate(zip(self.accel_columns, colors, labels)):
            axes[0, 0].plot(time, df_unfiltered[col].values, color=color, 
                          label=f'Accel {label}', linewidth=1.5, alpha=0.7)
        
        axes[0, 0].set_xlabel('Time (s)')
        axes[0, 0].set_ylabel('Acceleration (g)')
        axes[0, 0].set_title('Acceleration Data - Unfiltered')
        axes[0, 0].legend(loc='upper right')
        axes[0, 0].grid(True, alpha=0.3)
        
        # Filtered acceleration
        for i, (col, color, label) in enumerate(zip(self.accel_columns, colors, labels)):
            axes[0, 1].plot(time, df_filtered[col].values, color=color, 
                          label=f'Accel {label}', linewidth=1.5)
        
        axes[0, 1].set_xlabel('Time (s)')
        axes[0, 1].set_ylabel('Acceleration (g)')
        axes[0, 1].set_title(f'Acceleration Data - Filtered (fc={self.filter_cutoff}Hz, order={self.filter_order})')
        axes[0, 1].legend(loc='upper right')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Plot gyroscope data (filtered vs unfiltered)
        # Unfiltered gyroscope
        for i, (col, color, label) in enumerate(zip(self.gyro_columns, colors, labels)):
            axes[1, 0].plot(time, df_unfiltered[col].values, color=color, 
                          label=f'Gyro {label}', linewidth=1.5, alpha=0.7)
        
        axes[1, 0].set_xlabel('Time (s)')
        axes[1, 0].set_ylabel('Angular Velocity (°/s)')
        axes[1, 0].set_title('Gyroscope Data - Unfiltered')
        axes[1, 0].legend(loc='upper right')
        axes[1, 0].grid(True, alpha=0.3)
        
        # Filtered gyroscope
        for i, (col, color, label) in enumerate(zip(self.gyro_columns, colors, labels)):
            axes[1, 1].plot(time, df_filtered[col].values, color=color, 
                          label=f'Gyro {label}', linewidth=1.5)
        
        axes[1, 1].set_xlabel('Time (s)')
        axes[1, 1].set_ylabel('Angular Velocity (°/s)')
        axes[1, 1].set_title(f'Gyroscope Data - Filtered (fc={self.filter_cutoff}Hz, order={self.filter_order})')
        axes[1, 1].legend(loc='upper right')
        axes[1, 1].grid(True, alpha=0.3)
        
        # Add statistics
        if self.filter_enabled:
            # Calculate signal power reduction
            accel_power_reduction = []
            gyro_power_reduction = []
            
            for col in self.accel_columns:
                orig_power = np.mean(df_unfiltered[col].values**2)
                filt_power = np.mean(df_filtered[col].values**2)
                if orig_power > 0:
                    reduction = (1 - filt_power/orig_power) * 100
                    accel_power_reduction.append(reduction)
            
            for col in self.gyro_columns:
                orig_power = np.mean(df_unfiltered[col].values**2)
                filt_power = np.mean(df_filtered[col].values**2)
                if orig_power > 0:
                    reduction = (1 - filt_power/orig_power) * 100
                    gyro_power_reduction.append(reduction)
            
            avg_accel_reduction = np.mean(accel_power_reduction)
            avg_gyro_reduction = np.mean(gyro_power_reduction)
            
            filter_info = f"Filter: Butterworth Low-pass\\n"
            filter_info += f"Cutoff: {self.filter_cutoff} Hz\\n"
            filter_info += f"Order: {self.filter_order}\\n"
            filter_info += f"Accel noise reduction: {avg_accel_reduction:.1f}%\\n"
            filter_info += f"Gyro noise reduction: {avg_gyro_reduction:.1f}%"
            
            fig.text(0.02, 0.02, filter_info, fontsize=10, 
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        
        plt.tight_layout()
        plt.show()
        
        # Print summary
        print(f"\n=== Data Summary for {self.selected_device} ===")
        print(f"Data type: {data_type}")
        print(f"Duration: {time[-1]:.2f} seconds")
        print(f"Samples: {len(df_filtered)}")
        print(f"Sample rate: {self.sample_rate} Hz")
        if self.filter_enabled:
            print(f"Filter: Butterworth low-pass, cutoff={self.filter_cutoff}Hz, order={self.filter_order}")

def calibrate_imu_to_board(static_accel_data, rotation_gyro_data):
    """
    Calculate rotation matrix between IMU and snowboard
    
    Args:
        static_accel_data: Acceleration data during static placement (N×3 array, m/s²)
        rotation_gyro_data: Gyroscope data during rotation (M×3 array, rad/s)
    
    Returns:
        R_board_to_imu: 3×3 rotation matrix (transforms vectors from board to IMU frame)
        installation_angles: Installation angles (roll, pitch, yaw) in degrees
    """
    # Validate input data
    if np.any(np.isnan(static_accel_data)) or np.any(np.isinf(static_accel_data)):
        raise ValueError("Static acceleration data contains NaN or Inf values")
    
    if np.any(np.isnan(rotation_gyro_data)) or np.any(np.isinf(rotation_gyro_data)):
        raise ValueError("Rotation gyroscope data contains NaN or Inf values")
    
    # Step 1: Static Calibration
    avg_accel = np.mean(static_accel_data, axis=0)
    zb_imu = avg_accel / np.linalg.norm(avg_accel)
    
    print(f"Average acceleration: {avg_accel}")
    print(f"Board normal (zb) in IMU frame: {zb_imu}")
    
    # Step 2: Rotation Calibration
    U, s, Vt = svd(rotation_gyro_data, full_matrices=False)
    xb_imu = Vt[0]
    xb_imu = xb_imu / np.linalg.norm(xb_imu)
    
    print(f"Rotation axis (xb) in IMU frame: {xb_imu}")
    print(f"Singular values: {s}")
    
    # Orthogonalization
    dot_product = np.dot(zb_imu, xb_imu)
    if abs(dot_product) > 0.1:
        print(f"Warning: zb and xb not orthogonal (dot={dot_product:.3f}), performing orthogonalization")
        xb_imu = xb_imu - dot_product * zb_imu
        xb_imu = xb_imu / np.linalg.norm(xb_imu)
    
    # Calculate yb axis
    yb_imu = np.cross(zb_imu, xb_imu)
    yb_norm = np.linalg.norm(yb_imu)
    if yb_norm < 1e-6:
        raise ValueError("Cross product invalid, zb and xb may be parallel")
    yb_imu = yb_imu / yb_norm
    
    # Construct Rotation Matrix
    R_board_to_imu = np.column_stack((xb_imu, yb_imu, zb_imu))
    
    # Validate rotation matrix
    det = np.linalg.det(R_board_to_imu)
    if abs(det - 1) > 0.1:
        print(f"Warning: Rotation matrix determinant={det:.3f} (expected 1)")
    
    # Calculate installation angles
    rotation = Rotation.from_matrix(R_board_to_imu)
    installation_angles = rotation.as_euler('xyz', degrees=True)
    
    return R_board_to_imu, installation_angles

# Create data loader instance
data_loader = IMUDataLoader()

# Create UI components
sample_rate_widget = widgets.FloatText(
    value=30.0,
    min=1.0,
    max=1000.0,
    description='Sample Rate (Hz):',
    style={'description_width': 'initial'}
)

device_selector = widgets.Dropdown(
    options=[],
    description='Select Device:',
    style={'description_width': 'initial'},
    disabled=True
)

# Filter controls
filter_enabled_widget = widgets.Checkbox(
    value=True,
    description='Enable Low-pass Filter',
    style={'description_width': 'initial'}
)

filter_cutoff_widget = widgets.FloatSlider(
    value=10.0,
    min=0.1,
    max=50.0,
    step=0.1,
    description='Cutoff Frequency (Hz):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

filter_order_widget = widgets.IntSlider(
    value=4,
    min=1,
    max=8,
    step=1,
    description='Filter Order:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Static data upload widget
static_upload = widgets.FileUpload(
    accept='.csv,.txt',
    multiple=False,
    description='Select Static Data'
)
static_output = widgets.Output()

# Static data preview button
static_preview_btn = widgets.Button(
    description="Preview Static Data",
    button_style='primary',
    icon='eye',
    disabled=True
)

# Rotation data upload widget
rotation_upload = widgets.FileUpload(
    accept='.csv,.txt',
    multiple=False,
    description='Select Rotation Data'
)
rotation_output = widgets.Output()

# Rotation data preview button
rotation_preview_btn = widgets.Button(
    description="Preview Rotation Data",
    button_style='primary',
    icon='eye',
    disabled=True
)

# Calibration button
calibrate_btn = widgets.Button(description="Run Calibration", button_style='success')

# Status display
status_output = widgets.Output()

# Results display
results_output = widgets.Output()

def update_sample_rate(change):
    """Update sample rate"""
    data_loader.sample_rate = change['new']
    # Update filter cutoff slider max value
    filter_cutoff_widget.max = change['new'] / 2 - 0.1
    with status_output:
        clear_output()
        print(f"Sample rate updated to: {change['new']} Hz")
        print(f"Maximum filter cutoff: {filter_cutoff_widget.max:.1f} Hz")

def update_device_selection(change):
    """Update selected device"""
    data_loader.selected_device = change['new']
    with status_output:
        clear_output()
        print(f"Selected device: {change['new']}")

def update_filter_settings(change):
    """Update filter settings"""
    data_loader.filter_enabled = filter_enabled_widget.value
    data_loader.filter_cutoff = filter_cutoff_widget.value
    data_loader.filter_order = filter_order_widget.value
    
    # Enable/disable filter controls
    filter_cutoff_widget.disabled = not filter_enabled_widget.value
    filter_order_widget.disabled = not filter_enabled_widget.value
    
    with status_output:
        clear_output()
        if data_loader.filter_enabled:
            print(f"Filter enabled: Butterworth low-pass")
            print(f"Cutoff frequency: {data_loader.filter_cutoff} Hz")
            print(f"Filter order: {data_loader.filter_order}")
        else:
            print("Filter disabled")

def update_device_list():
    """Update device selector options"""
    all_devices = set()
    
    if data_loader.static_data is not None:
        devices = data_loader.detect_devices(data_loader.static_data)
        all_devices.update(devices)
    
    if data_loader.rotation_data is not None:
        devices = data_loader.detect_devices(data_loader.rotation_data)
        all_devices.update(devices)
    
    if all_devices:
        device_selector.options = list(all_devices)
        device_selector.disabled = False
        if data_loader.selected_device not in all_devices and all_devices:
            data_loader.selected_device = list(all_devices)[0]
            device_selector.value = data_loader.selected_device
    else:
        device_selector.options = []
        device_selector.disabled = True

def handle_static_upload(change):
    """Handle static data upload"""
    with static_output:
        clear_output()
        if len(static_upload.value) > 0:
            try:
                uploaded_file = static_upload.value[0]
                file_content = uploaded_file['content']
                
                result = data_loader.load_data(file_content, 'static')
                
                if result is not None:
                    update_device_list()
                    static_preview_btn.disabled = False  # Enable preview button
                    
                    with status_output:
                        clear_output()
                        print("✅ Static data loaded successfully!")
                        print(f"Total samples: {len(data_loader.static_data)}")
                        
                        if data_loader.selected_device:
                            df_filtered = data_loader.get_processed_data('static')
                            if df_filtered is not None and len(df_filtered) > 0:
                                print(f"Samples for {data_loader.selected_device}: {len(df_filtered)}")
                else:
                    static_preview_btn.disabled = True
                    with status_output:
                        clear_output()
                        print("❌ Failed to load static data")
                    
            except Exception as e:
                static_preview_btn.disabled = True
                with status_output:
                    clear_output()
                    print(f"❌ Error loading static data: {e}")

def handle_rotation_upload(change):
    """Handle rotation data upload"""
    with rotation_output:
        clear_output()
        if len(rotation_upload.value) > 0:
            try:
                uploaded_file = rotation_upload.value[0]
                file_content = uploaded_file['content']
                
                result = data_loader.load_data(file_content, 'rotation')
                
                if result is not None:
                    update_device_list()
                    rotation_preview_btn.disabled = False  # Enable preview button
                    
                    with status_output:
                        clear_output()
                        print("✅ Rotation data loaded successfully!")
                        print(f"Total samples: {len(data_loader.rotation_data)}")
                        
                        if data_loader.selected_device:
                            df_filtered = data_loader.get_processed_data('rotation')
                            if df_filtered is not None and len(df_filtered) > 0:
                                print(f"Samples for {data_loader.selected_device}: {len(df_filtered)}")
                else:
                    rotation_preview_btn.disabled = True
                    with status_output:
                        clear_output()
                        print("❌ Failed to load rotation data")
                    
            except Exception as e:
                rotation_preview_btn.disabled = True
                with status_output:
                    clear_output()
                    print(f"❌ Error loading rotation data: {e}")

def preview_static_data(b):
    """Preview static data"""
    with static_output:
        clear_output()
        if data_loader.static_data is not None and data_loader.selected_device:
            data_loader.preview_data('static')
        else:
            print("No static data or device selected")

def preview_rotation_data(b):
    """Preview rotation data"""
    with rotation_output:
        clear_output()
        if data_loader.rotation_data is not None and data_loader.selected_device:
            data_loader.preview_data('rotation')
        else:
            print("No rotation data or device selected")

def run_calibration(b):
    """Run calibration process and save results"""
    global calibration_results  # 确保使用全局变量
    
    with results_output:
        clear_output()
        
        # Check if both datasets are loaded
        if data_loader.static_data is None or data_loader.rotation_data is None:
            print("❌ Please load both static and rotation data first")
            return
        
        if data_loader.selected_device is None:
            print("❌ Please select a device first")
            return
            
        try:
            print(f"Starting calibration process for {data_loader.selected_device}...")
            if data_loader.filter_enabled:
                print(f"Using filtered data (cutoff={data_loader.filter_cutoff}Hz, order={data_loader.filter_order})")
            else:
                print("Using unfiltered data")
            
            # Get data for calibration
            static_accel = data_loader.get_static_accel()
            rotation_gyro = data_loader.get_rotation_gyro()
            
            if static_accel is None or rotation_gyro is None:
                print(f"❌ No data found for device: {data_loader.selected_device}")
                return
            
            if len(static_accel) == 0 or len(rotation_gyro) == 0:
                print(f"❌ Empty data for device: {data_loader.selected_device}")
                return
            
            print(f"Static acceleration data shape: {static_accel.shape}")
            print(f"Rotation gyroscope data shape: {rotation_gyro.shape}")
            
            # Run calibration
            R_board_to_imu, angles = calibrate_imu_to_board(static_accel, rotation_gyro)
            
            # ===== 保存校准结果 =====
            calibration_results[data_loader.selected_device] = {
                'R_board_to_imu': R_board_to_imu.copy(),
                'R_imu_to_board': R_board_to_imu.T.copy(),
                'angles': angles.copy(),
                'device_name': data_loader.selected_device,
                'filter_settings': {
                    'enabled': data_loader.filter_enabled,
                    'cutoff': data_loader.filter_cutoff,
                    'order': data_loader.filter_order
                }
            }
            
            # Display results
            print("\n=== Calibration Results ===")
            print(f"Device: {data_loader.selected_device}")
            print("Rotation Matrix (Board to IMU):")
            print(np.array2string(R_board_to_imu, precision=4, suppress_small=True))
            
            print("\nInstallation Angles (degrees):")
            print(f"Roll (X): {angles[0]:.2f}°")
            print(f"Pitch (Y): {angles[1]:.2f}°")
            print(f"Yaw (Z): {angles[2]:.2f}°")
            
            # 3D Visualization
            from mpl_toolkits.mplot3d.art3d import Poly3DCollection
            import matplotlib.patches as mpatches
            
            fig = plt.figure(figsize=(10, 8))
            ax = fig.add_subplot(111, projection='3d')
            
            # Define snowboard dimensions
            board_length = 1.6
            board_width = 0.3
            board_thickness = 0.05
            
            # Define IMU dimensions
            imu_size = 0.06
            imu_height = 0.02
            
            # IMU position on board
            imu_pos_board = np.array([0, 0, board_thickness/2 + imu_height/2 + 0.01])
            
            # Create snowboard vertices
            board_vertices = np.array([
                [-board_length/2, -board_width/2, -board_thickness/2],
                [board_length/2, -board_width/2, -board_thickness/2],
                [board_length/2, board_width/2, -board_thickness/2],
                [-board_length/2, board_width/2, -board_thickness/2],
                [-board_length/2, -board_width/2, board_thickness/2],
                [board_length/2, -board_width/2, board_thickness/2],
                [board_length/2, board_width/2, board_thickness/2],
                [-board_length/2, board_width/2, board_thickness/2]
            ])
            
            # Define faces of the board
            board_faces = [
                [board_vertices[0], board_vertices[1], board_vertices[5], board_vertices[4]],
                [board_vertices[1], board_vertices[2], board_vertices[6], board_vertices[5]],
                [board_vertices[2], board_vertices[3], board_vertices[7], board_vertices[6]],
                [board_vertices[3], board_vertices[0], board_vertices[4], board_vertices[7]],
                [board_vertices[4], board_vertices[5], board_vertices[6], board_vertices[7]],
                [board_vertices[3], board_vertices[2], board_vertices[1], board_vertices[0]]
            ]
            
            # Create IMU vertices
            imu_vertices_local = np.array([
                [-imu_size/2, -imu_size/2, -imu_height/2],
                [imu_size/2, -imu_size/2, -imu_height/2],
                [imu_size/2, imu_size/2, -imu_height/2],
                [-imu_size/2, imu_size/2, -imu_height/2],
                [-imu_size/2, -imu_size/2, imu_height/2],
                [imu_size/2, -imu_size/2, imu_height/2],
                [imu_size/2, imu_size/2, imu_height/2],
                [-imu_size/2, imu_size/2, imu_height/2]
            ])
            
            # Transform IMU vertices to board coordinate system
            R_imu_to_board = R_board_to_imu.T
            imu_vertices = np.array([R_imu_to_board @ v + imu_pos_board for v in imu_vertices_local])
            
            # Define faces of the IMU
            imu_faces = [
                [imu_vertices[0], imu_vertices[1], imu_vertices[5], imu_vertices[4]],
                [imu_vertices[1], imu_vertices[2], imu_vertices[6], imu_vertices[5]],
                [imu_vertices[2], imu_vertices[3], imu_vertices[7], imu_vertices[6]],
                [imu_vertices[3], imu_vertices[0], imu_vertices[4], imu_vertices[7]],
                [imu_vertices[4], imu_vertices[5], imu_vertices[6], imu_vertices[7]],
                [imu_vertices[3], imu_vertices[2], imu_vertices[1], imu_vertices[0]]
            ]
            
            # Plot snowboard
            board_collection = Poly3DCollection(board_faces, alpha=0.3, facecolor='#8B4513', edgecolor='#654321')
            ax.add_collection3d(board_collection)
            
            # Plot IMU
            imu_collection = Poly3DCollection(imu_faces, alpha=0.7, facecolor='#808080', edgecolor='#000000')
            ax.add_collection3d(imu_collection)
            
            # Plot coordinate axes
            axis_length = 0.5
            axis_width = 3
            
            # Board coordinate axes
            ax.quiver(0, 0, 0, axis_length, 0, 0, color='red', arrow_length_ratio=0.1, linewidth=axis_width)
            ax.quiver(0, 0, 0, 0, axis_length, 0, color='green', arrow_length_ratio=0.1, linewidth=axis_width)
            ax.quiver(0, 0, 0, 0, 0, axis_length, color='blue', arrow_length_ratio=0.1, linewidth=axis_width)
            
            # IMU coordinate axes
            xi_in_board = R_imu_to_board @ np.array([1, 0, 0])
            yi_in_board = R_imu_to_board @ np.array([0, 1, 0])
            zi_in_board = R_imu_to_board @ np.array([0, 0, 1])
            
            imu_axis_length = 0.3
            ax.quiver(imu_pos_board[0], imu_pos_board[1], imu_pos_board[2], 
                     xi_in_board[0]*imu_axis_length, xi_in_board[1]*imu_axis_length, xi_in_board[2]*imu_axis_length, 
                     color='red', arrow_length_ratio=0.15, linewidth=2, linestyle='--', alpha=0.7)
            ax.quiver(imu_pos_board[0], imu_pos_board[1], imu_pos_board[2], 
                     yi_in_board[0]*imu_axis_length, yi_in_board[1]*imu_axis_length, yi_in_board[2]*imu_axis_length, 
                     color='green', arrow_length_ratio=0.15, linewidth=2, linestyle='--', alpha=0.7)
            ax.quiver(imu_pos_board[0], imu_pos_board[1], imu_pos_board[2], 
                     zi_in_board[0]*imu_axis_length, zi_in_board[1]*imu_axis_length, zi_in_board[2]*imu_axis_length, 
                     color='blue', arrow_length_ratio=0.15, linewidth=2, linestyle='--', alpha=0.7)
            
            # Labels
            ax.text(axis_length*1.1, 0, 0, 'xb', color='red', fontsize=12, weight='bold')
            ax.text(0, axis_length*1.1, 0, 'yb', color='green', fontsize=12, weight='bold')
            ax.text(0, 0, axis_length*1.1, 'zb', color='blue', fontsize=12, weight='bold')
            
            # Set labels and title
            ax.set_xlabel('Board X (m)', fontsize=12)
            ax.set_ylabel('Board Y (m)', fontsize=12)
            ax.set_zlabel('Board Z (m)', fontsize=12)
            ax.set_title(f'IMU Installation on Snowboard - {data_loader.selected_device}', fontsize=14, weight='bold')
            
            # Legend
            board_patch = mpatches.Patch(color='#8B4513', alpha=0.3, label='Snowboard')
            imu_patch = mpatches.Patch(color='#808080', alpha=0.7, label=f'IMU ({data_loader.selected_device})')
            ax.legend(handles=[board_patch, imu_patch], loc='upper right')
            
            # Set view
            ax.set_box_aspect([1, 1, 0.3])
            ax.view_init(elev=25, azim=45)
            
            # Set axis limits
            max_dim = max(board_length, board_width) * 0.7
            ax.set_xlim([-max_dim, max_dim])
            ax.set_ylim([-max_dim, max_dim])
            ax.set_zlim([-0.2, 0.4])
            
            ax.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            # ===== 验证保存结果 =====
            print(f"\n✅ Calibration completed successfully!")
            print(f"✅ Calibration results saved for {data_loader.selected_device}")
            print(f"Total calibrated devices: {len(calibration_results)}")
            print(f"Calibrated devices: {list(calibration_results.keys())}")
            
        except Exception as e:
            print(f"❌ Calibration failed: {e}")
            import traceback
            traceback.print_exc()

# 添加查看校准结果的按钮
check_calibration_btn = widgets.Button(description="Check Calibration Status", button_style='info')

def check_calibration_status(b):
    """Check calibration status"""
    with status_output:
        clear_output()
        print("=== Calibration Status ===")
        print(f"Total calibrated devices: {len(calibration_results)}")
        if calibration_results:
            print("\nCalibrated devices:")
            for device, data in calibration_results.items():
                angles = data['angles']
                filter_settings = data.get('filter_settings', {})
                print(f"  {device}:")
                print(f"    Roll: {angles[0]:.2f}°, Pitch: {angles[1]:.2f}°, Yaw: {angles[2]:.2f}°")
                if filter_settings.get('enabled', False):
                    print(f"    Filter: {filter_settings['cutoff']}Hz, order {filter_settings['order']}")
        else:
            print("No devices calibrated yet.")

# Bind event handlers
sample_rate_widget.observe(update_sample_rate, names='value')
device_selector.observe(update_device_selection, names='value')
filter_enabled_widget.observe(update_filter_settings, names='value')
filter_cutoff_widget.observe(update_filter_settings, names='value')
filter_order_widget.observe(update_filter_settings, names='value')
static_upload.observe(handle_static_upload, names='value')
rotation_upload.observe(handle_rotation_upload, names='value')
static_preview_btn.on_click(preview_static_data)
rotation_preview_btn.on_click(preview_rotation_data)
calibrate_btn.on_click(run_calibration)
check_calibration_btn.on_click(check_calibration_status)

# Display UI
print("# 雪板IMU校准系统 - 带低通滤波功能")
print("上传静态和旋转IMU数据文件进行校准")

# Basic settings
display(widgets.HBox([sample_rate_widget, device_selector]))

# Filter settings
print("\n## 滤波设置")
display(filter_enabled_widget)
display(filter_cutoff_widget)
display(filter_order_widget)

print("\n## 1. 上传静态数据")
print("雪板水平放置时的数据")
display(widgets.VBox([
    static_upload,
    static_preview_btn,
    static_output
]))

print("\n## 2. 上传旋转数据")
print("绕雪板前进方向旋转时的数据")
display(widgets.VBox([
    rotation_upload,
    rotation_preview_btn,
    rotation_output
]))

print("\n## 3. 运行校准")
display(widgets.HBox([calibrate_btn, check_calibration_btn]))

print("\n## 状态信息")
display(status_output)

print("\n## 校准结果")
display(results_output)

# 初始检查
check_calibration_status(None)
# Initialize filter settings
update_filter_settings(None)

# 雪板IMU校准系统 - 带低通滤波功能
上传静态和旋转IMU数据文件进行校准



## 滤波设置


Checkbox(value=True, description='Enable Low-pass Filter', style=CheckboxStyle(description_width='initial'))

FloatSlider(value=10.0, description='Cutoff Frequency (Hz):', layout=Layout(width='400px'), max=50.0, min=0.1,…

IntSlider(value=4, description='Filter Order:', layout=Layout(width='400px'), max=8, min=1, style=SliderStyle(…

...


## 3. 运行校准



## 状态信息


Output()


## 校准结果


Output()

In [34]:
# ========== Complete Motion Data Analysis Module with Filtering ==========
# This code should be added after the calibration code

print("\n\n# ========== Motion Data Analysis Module ==========")
print("Analyze IMU motion data using calibration results")

class MotionDataAnalyzer:
    def __init__(self, calibration_results):
        self.calibration_results = calibration_results
        self.motion_data = {}  # Raw motion data for each device
        self.board_motion = {}  # Data converted to board frame
        self.sample_rate = 30.0
        
        # Filter settings
        self.filter_enabled = True
        self.filter_cutoff = 10.0  # Hz
        self.filter_order = 4
        
    def apply_lowpass_filter(self, data, cutoff_freq, order):
        """Apply Butterworth lowpass filter to data"""
        if not self.filter_enabled or cutoff_freq >= self.sample_rate / 2:
            return data
            
        # Design Butterworth filter
        nyquist = self.sample_rate / 2
        normalized_cutoff = cutoff_freq / nyquist
        b, a = signal.butter(order, normalized_cutoff, btype='low', analog=False)
        
        # Apply filter to each column
        filtered_data = np.zeros_like(data)
        for i in range(data.shape[1]):
            # Apply zero-phase filtering (filtfilt) to avoid phase shift
            filtered_data[:, i] = signal.filtfilt(b, a, data[:, i])
        
        return filtered_data
        
    def load_motion_data(self, file_content):
        """Load motion data file"""
        try:
            # Create temporary data loader
            temp_loader = IMUDataLoader()
            temp_loader.sample_rate = self.sample_rate
            temp_loader.accel_columns = data_loader.accel_columns
            temp_loader.gyro_columns = data_loader.gyro_columns
            temp_loader.device_column = data_loader.device_column
            temp_loader.g_to_ms2 = data_loader.g_to_ms2
            
            # Copy filter settings
            temp_loader.filter_enabled = self.filter_enabled
            temp_loader.filter_cutoff = self.filter_cutoff
            temp_loader.filter_order = self.filter_order
            
            # Load data
            df = temp_loader.load_data(file_content, 'motion')
            if df is not None:
                # Get all devices
                devices = temp_loader.detect_devices(df)
                print(f"Found devices in motion data: {devices}")
                
                # Process each device
                processed_count = 0
                for device in devices:
                    if device in self.calibration_results:
                        device_df = temp_loader.filter_by_device(df, device)
                        device_df = temp_loader.check_and_clean_data(device_df, 'motion')
                        self.process_device_motion(device_df, device, temp_loader)
                        processed_count += 1
                    else:
                        print(f"Warning: No calibration data for {device}, skipping...")
                
                if processed_count > 0:
                    print(f"Successfully processed {processed_count} device(s)")
                    if self.filter_enabled:
                        print(f"Applied filter: Butterworth low-pass, cutoff={self.filter_cutoff}Hz, order={self.filter_order}")
                    return True
                else:
                    print("Error: No devices could be processed")
                    return False
            return False
        except Exception as e:
            print(f"Error loading motion data: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    def process_device_motion(self, df, device_name, data_loader):
        """Process motion data for a single device"""
        # Get calibration parameters
        calib = self.calibration_results[device_name]
        R_imu_to_board = calib['R_imu_to_board']
        
        # Add timestamp
        num_samples = len(df)
        time_array = np.arange(0, num_samples / self.sample_rate, 1 / self.sample_rate)[:num_samples]
        
        # Get IMU data
        accel_imu_raw = df[data_loader.accel_columns].values * data_loader.g_to_ms2  # m/s²
        gyro_imu_raw = df[data_loader.gyro_columns].values * (np.pi / 180.0)  # rad/s
        
        # Apply filter if enabled
        if self.filter_enabled:
            accel_imu = self.apply_lowpass_filter(accel_imu_raw, self.filter_cutoff, self.filter_order)
            gyro_imu = self.apply_lowpass_filter(gyro_imu_raw, self.filter_cutoff, self.filter_order)
        else:
            accel_imu = accel_imu_raw
            gyro_imu = gyro_imu_raw
        
        # Convert to board frame
        accel_board = np.array([R_imu_to_board @ a for a in accel_imu])
        gyro_board = np.array([R_imu_to_board @ g for g in gyro_imu])
        
        # Store results
        self.board_motion[device_name] = {
            'time': time_array,
            'accel_board': accel_board,
            'gyro_board': gyro_board,
            'accel_imu_raw': accel_imu_raw,  # Store raw data for comparison
            'gyro_imu_raw': gyro_imu_raw,
            'device_name': device_name
        }
        
        print(f"  Processed {device_name}: {len(time_array)} samples")
    
    def quaternion_multiply(self, q1, q2):
        """Quaternion multiplication"""
        w1, x1, y1, z1 = q1
        w2, x2, y2, z2 = q2
        
        return np.array([
            w1*w2 - x1*x2 - y1*y2 - z1*z2,
            w1*x2 + x1*w2 + y1*z2 - z1*y2,
            w1*y2 - x1*z2 + y1*w2 + z1*x2,
            w1*z2 + x1*y2 - y1*x2 + z1*w2
        ])
    
    def plot_filter_comparison(self):
        """Plot comparison of filtered vs unfiltered data"""
        if len(self.board_motion) == 0:
            print("No motion data loaded")
            return
            
        device = list(self.board_motion.keys())[0]
        data = self.board_motion[device]
        
        # Create figure
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        fig.suptitle(f'Filter Effect Comparison - {device}', fontsize=14, weight='bold')
        
        time = data['time']
        
        # Raw vs filtered acceleration magnitude
        accel_raw_mag = np.linalg.norm(data['accel_imu_raw'], axis=1)
        accel_board_mag = np.linalg.norm(data['accel_board'], axis=1)
        
        axes[0, 0].plot(time, accel_raw_mag, 'b-', alpha=0.6, label='Raw IMU', linewidth=1)
        axes[0, 0].plot(time, accel_board_mag, 'r-', label='Filtered & Transformed', linewidth=2)
        axes[0, 0].set_xlabel('Time (s)')
        axes[0, 0].set_ylabel('Acceleration Magnitude (m/s²)')
        axes[0, 0].set_title('Acceleration Magnitude')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Raw vs filtered gyro magnitude
        gyro_raw_mag = np.linalg.norm(data['gyro_imu_raw'], axis=1) * 180/np.pi
        gyro_board_mag = np.linalg.norm(data['gyro_board'], axis=1) * 180/np.pi
        
        axes[0, 1].plot(time, gyro_raw_mag, 'b-', alpha=0.6, label='Raw IMU', linewidth=1)
        axes[0, 1].plot(time, gyro_board_mag, 'r-', label='Filtered & Transformed', linewidth=2)
        axes[0, 1].set_xlabel('Time (s)')
        axes[0, 1].set_ylabel('Angular Velocity Magnitude (°/s)')
        axes[0, 1].set_title('Angular Velocity Magnitude')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Frequency spectrum of acceleration
        from scipy.fft import fft, fftfreq
        
        n = len(time)
        freqs = fftfreq(n, 1/self.sample_rate)[:n//2]
        
        accel_raw_fft = np.abs(fft(accel_raw_mag)[:n//2])
        accel_filt_fft = np.abs(fft(accel_board_mag)[:n//2])
        
        axes[1, 0].semilogy(freqs, accel_raw_fft, 'b-', alpha=0.6, label='Raw', linewidth=1)
        axes[1, 0].semilogy(freqs, accel_filt_fft, 'r-', label='Filtered', linewidth=2)
        if self.filter_enabled:
            axes[1, 0].axvline(x=self.filter_cutoff, color='g', linestyle='--', 
                              label=f'Cutoff: {self.filter_cutoff}Hz')
        axes[1, 0].set_xlabel('Frequency (Hz)')
        axes[1, 0].set_ylabel('Magnitude')
        axes[1, 0].set_title('Acceleration Frequency Spectrum')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        axes[1, 0].set_xlim([0, self.sample_rate/2])
        
        # Frequency spectrum of gyroscope
        gyro_raw_fft = np.abs(fft(gyro_raw_mag)[:n//2])
        gyro_filt_fft = np.abs(fft(gyro_board_mag)[:n//2])
        
        axes[1, 1].semilogy(freqs, gyro_raw_fft, 'b-', alpha=0.6, label='Raw', linewidth=1)
        axes[1, 1].semilogy(freqs, gyro_filt_fft, 'r-', label='Filtered', linewidth=2)
        if self.filter_enabled:
            axes[1, 1].axvline(x=self.filter_cutoff, color='g', linestyle='--', 
                              label=f'Cutoff: {self.filter_cutoff}Hz')
        axes[1, 1].set_xlabel('Frequency (Hz)')
        axes[1, 1].set_ylabel('Magnitude')
        axes[1, 1].set_title('Gyroscope Frequency Spectrum')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].set_xlim([0, self.sample_rate/2])
        
        plt.tight_layout()
        plt.show()
        
        # Print statistics
        print(f"\n=== Filter Statistics ===")
        if self.filter_enabled:
            print(f"Filter: Butterworth low-pass")
            print(f"Cutoff frequency: {self.filter_cutoff} Hz")
            print(f"Filter order: {self.filter_order}")
            
            # Calculate noise reduction
            accel_noise = np.std(accel_raw_mag - accel_board_mag)
            accel_reduction = (1 - np.std(accel_board_mag) / np.std(accel_raw_mag)) * 100
            print(f"\nAcceleration noise reduction: {accel_reduction:.1f}%")
            
            gyro_noise = np.std(gyro_raw_mag - gyro_board_mag)
            gyro_reduction = (1 - np.std(gyro_board_mag) / np.std(gyro_raw_mag)) * 100
            print(f"Gyroscope noise reduction: {gyro_reduction:.1f}%")
        else:
            print("Filter disabled")
    
    def plot_imu_comparison(self):
        """Compare Euler angles and angular velocities between two IMUs"""
        if len(self.board_motion) < 2:
            print("Need at least 2 IMUs for comparison")
            return
        
        # Get first two devices
        devices = list(self.board_motion.keys())[:2]
        colors = ['blue', 'red']
        
        # Calculate Euler angles for each IMU
        euler_angles_dict = {}
        
        for device in devices:
            data = self.board_motion[device]
            gyro_board = data['gyro_board']
            time = data['time']
            
            # Use quaternion integration
            from scipy.spatial.transform import Rotation as R
            
            dt = 1.0 / self.sample_rate
            euler_angles = []
            
            # Initial attitude
            q = np.array([1.0, 0.0, 0.0, 0.0])  # [w, x, y, z]
            euler_angles.append([0.0, 0.0, 0.0])
            
            # Integrate angular velocity
            for i in range(1, len(time)):
                omega = gyro_board[i]
                omega_norm = np.linalg.norm(omega)
                
                if omega_norm > 1e-10:
                    axis = omega / omega_norm
                    angle = omega_norm * dt
                    dq = np.array([
                        np.cos(angle/2),
                        axis[0] * np.sin(angle/2),
                        axis[1] * np.sin(angle/2),
                        axis[2] * np.sin(angle/2)
                    ])
                else:
                    dq = np.array([1.0, 0.0, 0.0, 0.0])
                
                # Update quaternion
                q = self.quaternion_multiply(q, dq)
                q = q / np.linalg.norm(q)
                
                # Convert to Euler angles
                r = R.from_quat([q[1], q[2], q[3], q[0]])
                euler = r.as_euler('xyz', degrees=True)
                euler_angles.append(euler)
            
            euler_angles_dict[device] = np.array(euler_angles)
        
        # Create plots
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        fig.suptitle('Dual IMU Comparison - Board Frame', fontsize=16)
        
        # Row 1: Euler angles
        # Roll
        axes[0, 0].plot(self.board_motion[devices[0]]['time'], 
                        -euler_angles_dict[devices[0]][:, 0], 
                        color=colors[0], label=devices[0][:30], linewidth=2)
        axes[0, 0].plot(self.board_motion[devices[1]]['time'], 
                        euler_angles_dict[devices[1]][:, 0], 
                        color=colors[1], label=devices[1][:30], linewidth=2, alpha=0.8)
        axes[0, 0].set_xlabel('Time (s)')
        axes[0, 0].set_ylabel('Roll (deg)')
        axes[0, 0].set_title('Roll Angle')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Pitch
        axes[0, 1].plot(self.board_motion[devices[0]]['time'], 
                        euler_angles_dict[devices[0]][:, 1], 
                        color=colors[0], label=devices[0][:30], linewidth=2)
        axes[0, 1].plot(self.board_motion[devices[1]]['time'], 
                        euler_angles_dict[devices[1]][:, 1], 
                        color=colors[1], label=devices[1][:30], linewidth=2, alpha=0.8)
        axes[0, 1].set_xlabel('Time (s)')
        axes[0, 1].set_ylabel('Pitch (deg)')
        axes[0, 1].set_title('Pitch Angle')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Yaw
        axes[0, 2].plot(self.board_motion[devices[0]]['time'], 
                        euler_angles_dict[devices[0]][:, 2], 
                        color=colors[0], label=devices[0][:30], linewidth=2)
        axes[0, 2].plot(self.board_motion[devices[1]]['time'], 
                        euler_angles_dict[devices[1]][:, 2], 
                        color=colors[1], label=devices[1][:30], linewidth=2, alpha=0.8)
        axes[0, 2].set_xlabel('Time (s)')
        axes[0, 2].set_ylabel('Yaw (deg)')
        axes[0, 2].set_title('Yaw Angle')
        axes[0, 2].legend()
        axes[0, 2].grid(True, alpha=0.3)
        
        # Row 2: Angular velocities
        # X-axis
        axes[1, 0].plot(self.board_motion[devices[0]]['time'], 
                        self.board_motion[devices[0]]['gyro_board'][:, 0] * 180/np.pi, 
                        color=colors[0], label=devices[0][:30], linewidth=2)
        axes[1, 0].plot(self.board_motion[devices[1]]['time'], 
                        self.board_motion[devices[1]]['gyro_board'][:, 0] * 180/np.pi, 
                        color=colors[1], label=devices[1][:30], linewidth=2, alpha=0.8)
        axes[1, 0].set_xlabel('Time (s)')
        axes[1, 0].set_ylabel('omega_x (deg/s)')
        axes[1, 0].set_title('X-axis Angular Velocity')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # Y-axis
        axes[1, 1].plot(self.board_motion[devices[0]]['time'], 
                        self.board_motion[devices[0]]['gyro_board'][:, 1] * 180/np.pi, 
                        color=colors[0], label=devices[0][:30], linewidth=2)
        axes[1, 1].plot(self.board_motion[devices[1]]['time'], 
                        self.board_motion[devices[1]]['gyro_board'][:, 1] * 180/np.pi, 
                        color=colors[1], label=devices[1][:30], linewidth=2, alpha=0.8)
        axes[1, 1].set_xlabel('Time (s)')
        axes[1, 1].set_ylabel('omega_y (deg/s)')
        axes[1, 1].set_title('Y-axis Angular Velocity')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        # Z-axis
        axes[1, 2].plot(self.board_motion[devices[0]]['time'], 
                        self.board_motion[devices[0]]['gyro_board'][:, 2] * 180/np.pi, 
                        color=colors[0], label=devices[0][:30], linewidth=2)
        axes[1, 2].plot(self.board_motion[devices[1]]['time'], 
                        self.board_motion[devices[1]]['gyro_board'][:, 2] * 180/np.pi, 
                        color=colors[1], label=devices[1][:30], linewidth=2, alpha=0.8)
        axes[1, 2].set_xlabel('Time (s)')
        axes[1, 2].set_ylabel('omega_z (deg/s)')
        axes[1, 2].set_title('Z-axis Angular Velocity')
        axes[1, 2].legend()
        axes[1, 2].grid(True, alpha=0.3)
        
        # Add filter info to the plot
        if self.filter_enabled:
            filter_text = f"Filter: Butterworth LP, fc={self.filter_cutoff}Hz, order={self.filter_order}"
            fig.text(0.5, 0.94, filter_text, ha='center', fontsize=10, 
                    bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
        
        plt.tight_layout()
        plt.show()
        
        # Statistical analysis
        print("\n=== Statistical Analysis ===")
        print(f"Device 1: {devices[0]}")
        print(f"Device 2: {devices[1]}")
        print(f"Duration: {self.board_motion[devices[0]]['time'][-1]:.2f} seconds")
        print(f"Sample rate: {self.sample_rate} Hz")
        if self.filter_enabled:
            print(f"Filter: Butterworth low-pass, cutoff={self.filter_cutoff}Hz, order={self.filter_order}")
        print("")
        
        # Ensure same length for comparison
        min_len = min(len(euler_angles_dict[devices[0]]), len(euler_angles_dict[devices[1]]))
        
        # Euler angle differences
        euler_diff = euler_angles_dict[devices[0]][:min_len] - euler_angles_dict[devices[1]][:min_len]
        
        print("Euler Angle Statistics:")
        print("  RMS Differences:")
        print(f"    Roll:  {np.sqrt(np.mean(euler_diff[:, 0]**2)):.2f} deg")
        print(f"    Pitch: {np.sqrt(np.mean(euler_diff[:, 1]**2)):.2f} deg")
        print(f"    Yaw:   {np.sqrt(np.mean(euler_diff[:, 2]**2)):.2f} deg")
        
        print("\n  Maximum Differences:")
        print(f"    Roll:  {np.max(np.abs(euler_diff[:, 0])):.2f} deg")
        print(f"    Pitch: {np.max(np.abs(euler_diff[:, 1])):.2f} deg")
        print(f"    Yaw:   {np.max(np.abs(euler_diff[:, 2])):.2f} deg")
        
        print("\n  Mean Differences:")
        print(f"    Roll:  {np.mean(euler_diff[:, 0]):.2f} deg")
        print(f"    Pitch: {np.mean(euler_diff[:, 1]):.2f} deg")
        print(f"    Yaw:   {np.mean(euler_diff[:, 2]):.2f} deg")
        
        # Angular velocity differences
        gyro1 = self.board_motion[devices[0]]['gyro_board'][:min_len]
        gyro2 = self.board_motion[devices[1]]['gyro_board'][:min_len]
        gyro_diff = (gyro1 - gyro2) * 180/np.pi
        
        print("\nAngular Velocity Statistics:")
        print("  RMS Differences:")
        print(f"    omega_x: {np.sqrt(np.mean(gyro_diff[:, 0]**2)):.2f} deg/s")
        print(f"    omega_y: {np.sqrt(np.mean(gyro_diff[:, 1]**2)):.2f} deg/s")
        print(f"    omega_z: {np.sqrt(np.mean(gyro_diff[:, 2]**2)):.2f} deg/s")
        
        print("\n  Maximum Differences:")
        print(f"    omega_x: {np.max(np.abs(gyro_diff[:, 0])):.2f} deg/s")
        print(f"    omega_y: {np.max(np.abs(gyro_diff[:, 1])):.2f} deg/s")
        print(f"    omega_z: {np.max(np.abs(gyro_diff[:, 2])):.2f} deg/s")
        
        # Correlation analysis
        print("\nCorrelation Analysis:")
        for i, axis in enumerate(['X', 'Y', 'Z']):
            euler_corr = np.corrcoef(euler_angles_dict[devices[0]][:min_len, i], 
                                     euler_angles_dict[devices[1]][:min_len, i])[0, 1]
            gyro_corr = np.corrcoef(gyro1[:, i], gyro2[:, i])[0, 1]
            print(f"  {axis}-axis:")
            print(f"    Euler angle correlation: {euler_corr:.3f}")
            print(f"    Angular velocity correlation: {gyro_corr:.3f}")

# Create motion analyzer instance
if 'calibration_results' in globals() and len(calibration_results) > 0:
    motion_analyzer = MotionDataAnalyzer(calibration_results)
    print(f"Motion analyzer initialized with {len(calibration_results)} calibrated device(s)")
else:
    print("Error: No calibration results found. Please complete calibration first.")
    motion_analyzer = None

# Create UI components
motion_sample_rate = widgets.FloatText(
    value=30.0,
    min=1.0,
    max=1000.0,
    description='Sample Rate (Hz):',
    style={'description_width': 'initial'}
)

# Motion filter controls
motion_filter_enabled = widgets.Checkbox(
    value=True,
    description='Enable Low-pass Filter',
    style={'description_width': 'initial'}
)

motion_filter_cutoff = widgets.FloatSlider(
    value=10.0,
    min=0.1,
    max=50.0,
    step=0.1,
    description='Cutoff Frequency (Hz):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

motion_filter_order = widgets.IntSlider(
    value=4,
    min=1,
    max=8,
    step=1,
    description='Filter Order:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

motion_upload = widgets.FileUpload(
    accept='.csv,.txt',
    multiple=False,
    description='Select Motion Data'
)

motion_output = widgets.Output()

# Analysis buttons
analyze_btn = widgets.Button(
    description="Analyze Motion", 
    button_style='primary',
    icon='bar-chart'
)

filter_analysis_btn = widgets.Button(
    description="Show Filter Effect", 
    button_style='info',
    icon='filter'
)

# Results output
motion_results = widgets.Output()

def update_motion_sample_rate(change):
    """Update motion data sample rate"""
    if motion_analyzer:
        motion_analyzer.sample_rate = change['new']
        # Update filter cutoff slider max value
        motion_filter_cutoff.max = change['new'] / 2 - 0.1
    with motion_output:
        clear_output()
        print(f"Sample rate updated to: {change['new']} Hz")
        print(f"Maximum filter cutoff: {motion_filter_cutoff.max:.1f} Hz")

def update_motion_filter(change):
    """Update motion filter settings"""
    if motion_analyzer:
        motion_analyzer.filter_enabled = motion_filter_enabled.value
        motion_analyzer.filter_cutoff = motion_filter_cutoff.value
        motion_analyzer.filter_order = motion_filter_order.value
        
    # Enable/disable filter controls
    motion_filter_cutoff.disabled = not motion_filter_enabled.value
    motion_filter_order.disabled = not motion_filter_enabled.value
    
    with motion_output:
        clear_output()
        if motion_analyzer and motion_analyzer.filter_enabled:
            print(f"Filter enabled: Butterworth low-pass")
            print(f"Cutoff frequency: {motion_analyzer.filter_cutoff} Hz")
            print(f"Filter order: {motion_analyzer.filter_order}")
        else:
            print("Filter disabled")

def handle_motion_upload(change):
    """Handle motion data upload"""
    with motion_output:
        clear_output()
        if len(motion_upload.value) > 0 and motion_analyzer is not None:
            try:
                uploaded_file = motion_upload.value[0]
                file_content = uploaded_file['content']
                
                print("Loading motion data...")
                success = motion_analyzer.load_motion_data(file_content)
                
                if success:
                    print("\nMotion data loaded successfully!")
                    print(f"Devices with data: {list(motion_analyzer.board_motion.keys())}")
                else:
                    print("Failed to process motion data")
                    
            except Exception as e:
                print(f"Error: {e}")
                import traceback
                traceback.print_exc()

def analyze_motion(b):
    """Analyze motion data"""
    with motion_results:
        clear_output()
        if motion_analyzer and len(motion_analyzer.board_motion) >= 2:
            print("Analyzing motion data...")
            motion_analyzer.plot_imu_comparison()
        else:
            if motion_analyzer is None:
                print("Please complete calibration first")
            elif len(motion_analyzer.board_motion) == 0:
                print("Please load motion data first")
            else:
                print("Need at least 2 IMUs for comparison")

def show_filter_effect(b):
    """Show filter effect analysis"""
    with motion_results:
        clear_output()
        if motion_analyzer and len(motion_analyzer.board_motion) > 0:
            print("Analyzing filter effect...")
            motion_analyzer.plot_filter_comparison()
        else:
            if motion_analyzer is None:
                print("Please complete calibration first")
            else:
                print("Please load motion data first")

# Bind event handlers
motion_sample_rate.observe(update_motion_sample_rate, names='value')
motion_filter_enabled.observe(update_motion_filter, names='value')
motion_filter_cutoff.observe(update_motion_filter, names='value')
motion_filter_order.observe(update_motion_filter, names='value')
motion_upload.observe(handle_motion_upload, names='value')
analyze_btn.on_click(analyze_motion)
filter_analysis_btn.on_click(show_filter_effect)

# Display UI
print("\n## Motion Data Settings")
display(motion_sample_rate)

print("\n## Motion Filter Settings")
display(motion_filter_enabled)
display(motion_filter_cutoff)
display(motion_filter_order)

print("\n## Upload Motion Data")
print("Upload file containing IMU motion data")
display(widgets.VBox([motion_upload, motion_output]))

print("\n## Analysis")
display(widgets.HBox([analyze_btn, filter_analysis_btn]))

print("\n## Results")
display(motion_results)

# Status check
if 'calibration_results' in globals() and len(calibration_results) > 0:
    print(f"\nCalibrated devices available: {list(calibration_results.keys())}")
    print("Ready to analyze motion data")
else:
    print("\nNo calibration data found. Please complete IMU calibration first.")
    
# Initialize filter settings
update_motion_filter(None)



# ========== Motion Data Analysis Module ==========
Analyze IMU motion data using calibration results
Motion analyzer initialized with 1 calibrated device(s)

## Motion Data Settings


FloatText(value=30.0, description='Sample Rate (Hz):', style=DescriptionStyle(description_width='initial'))


## Motion Filter Settings


Checkbox(value=True, description='Enable Low-pass Filter', style=CheckboxStyle(description_width='initial'))

FloatSlider(value=10.0, description='Cutoff Frequency (Hz):', layout=Layout(width='400px'), max=50.0, min=0.1,…

...


## Upload Motion Data
Upload file containing IMU motion data



## Analysis



## Results


Output()


Calibrated devices available: ['WTLEFT(C3:11:EA:D7:53:77)']
Ready to analyze motion data
